<a href="https://colab.research.google.com/github/ananyascodehq/semantic-chatbot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install sentence-transformers


In [27]:
with open("dialogs.txt", "r", encoding="utf-8") as f:
    for _ in range(10):
        print(f.readline())


hi, how are you doing?	i'm fine. how about yourself?

i'm fine. how about yourself?	i'm pretty good. thanks for asking.

i'm pretty good. thanks for asking.	no problem. so how have you been?

no problem. so how have you been?	i've been great. what about you?

i've been great. what about you?	i've been good. i'm in school right now.

i've been good. i'm in school right now.	what school do you go to?

what school do you go to?	i go to pcc.

i go to pcc.	do you like it there?

do you like it there?	it's okay. it's a really big campus.

it's okay. it's a really big campus.	good luck with school.



In [28]:
import pandas as pd

pairs = []
with open("dialogs.txt", "r", encoding="utf-8") as f:
    for line in f:
        if "\t" in line:
            q, a = line.strip().split("\t")
            pairs.append((q, a))

df = pd.DataFrame(pairs, columns=["question", "answer"])
print("Total dialogs:", len(df))


Total dialogs: 3725


In [29]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [30]:
question_embeddings = model.encode(
    df["question"].tolist(),
    show_progress_bar=True
)


Batches:   0%|          | 0/117 [00:00<?, ?it/s]

In [35]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def semantic_chatbot_reply(user_input):
    user_embedding = model.encode([user_input])
    similarities = cosine_similarity(user_embedding, question_embeddings)[0]

    best_idx = similarities.argmax()
    best_score = similarities[best_idx]

    # Dynamic threshold
    word_count = len(user_input.split())
    threshold = 0.45 if word_count <= 2 else 0.6

    if best_score < threshold:
        return "Sorry, I didn't understand that."

    return df.iloc[best_idx]["answer"]



In [36]:
print("Semantic Chatbot running. Type 'exit' to stop.\n")
print("Responses are limited to existing dataset answers, since the chatbot is retrieval-based and not generative.\n")
while True:
    user_input = input("You: ").strip()

    if user_input.lower() in {"exit", "quit", "bye", "tata"}:
        print("Bot: Goodbye.")
        break

    if not user_input:
        print("Bot: Say something.")
        continue

    response = semantic_chatbot_reply(user_input)
    print("Bot:", response)


Semantic Chatbot running. Type 'exit' to stop.

Responses are limited to existing dataset answers, since the chatbot is retrieval-based and not generative.

You: hi
Bot: i'm fine. how about yourself?
You: hello
Bot: i'm fine. how about yourself?
You: whats up
Bot: nothing, how about you?
You: Nothing much
Bot: what about the baseball game?
You: we won
Bot: i wish i was free that night. i'm kind of mad that i didn't go.
You: i know
Bot: but you're doing 75.
You: ok
Bot: i hope you feel better.
You: bye
Bot: Goodbye.
